# Install requirements

In [1]:
%pip install spacy pandas
%conda install -c conda-forge ipywidgets

Note: you may need to restart the kernel to use updated packages.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Pipeline

In [3]:
import spacy
from spacy.lang.nl import Dutch

SPACY_MODEL = "nl_core_news_sm"

try:
    nlp = spacy.load(SPACY_MODEL)
except OSError:
    spacy.cli.download(SPACY_MODEL)
    nlp = spacy.load(SPACY_MODEL)

nlp.add_pipe("sentencizer")

for component in [
    "tok2vec",
    "morphologizer",
    "tagger",
    "parser",
    "lemmatizer",
    "attribute_ruler",
    "ner",
]:
    nlp.remove_pipe(component)


In [4]:
nlp.pipe_names

['sentencizer']

# Corpus Location

In [5]:
from pathlib import Path


DATA_DIR = Path.home() / "Documents" / "SemanticsOfSustainability" / "data" / "Joris"
assert DATA_DIR.is_dir()


## Read ANP

In [6]:
ANP_DIR = DATA_DIR / "ANP" / "CleanFiles_perYear"
assert ANP_DIR.is_dir()

ANP_FILE_GLOB = "ANP_19[0-9][0-9].csv.gz"
ANP_ENCODING = "iso8859_15"


### Count Sentences and Tokens

In [8]:
from tqdm import tqdm

n_sentences = 0
n_tokens = 0
n_docs = 0

files = list(ANP_DIR.glob(ANP_FILE_GLOB))


for f in tqdm(files, unit="file", position=0):
    df = pd.read_csv(
        f, compression="gzip", sep=";", encoding=ANP_ENCODING, parse_dates=["date"]
    ).dropna(subset="content")

    for index, value in tqdm(df.content.items(), unit="doc", total=len(df), position=1):
        n_docs += 1
        doc = nlp(value)
        for sent in doc.sents:
            n_sentences += 1
            for token in sent:
                n_tokens += 1

print(f"Number of documents:\t{n_docs}")
print(f"Number of sentences:\t{n_sentences}")
print(f"Number of tokens:\t{n_tokens}")


100%|██████████| 28067/28067 [00:35<00:00, 779.89doc/s]
0doc [00:00, ?doc/s]8 [00:36<28:27, 36.34s/file]
100%|██████████| 31069/31069 [00:45<00:00, 680.49doc/s]
0doc [00:00, ?doc/s]8 [03:36<32:37, 45.53s/file]
100%|██████████| 50049/50049 [01:06<00:00, 749.98doc/s]
0doc [00:00, ?doc/s]48 [14:27<21:49, 45.16s/file]
100%|██████████| 28668/28668 [00:39<00:00, 722.18doc/s]
0doc [00:00, ?doc/s]48 [21:30<15:41, 49.56s/file]
100%|██████████| 54564/54564 [01:11<00:00, 766.79doc/s]
0doc [00:00, ?doc/s]48 [24:55<13:40, 54.69s/file]
100%|██████████| 3151/3151 [00:10<00:00, 293.51doc/s]
0doc [00:00, ?doc/s]48 [31:50<03:52, 46.46s/file]
100%|██████████| 48/48 [35:05<00:00, 43.86s/file]

Number of documents:	: 1474649
Number of sentences:	: 23880052
Number of tokens:	: 226754135


## Staten Generaal

In [11]:
SG_DIR = DATA_DIR / "StatenGeneraal"

assert SG_DIR.is_dir()

SG_GLOB = "StatenGeneraal_[12][890][0-9][0-9].csv.gz"
SG_ENCODING = "iso8859_15"


In [15]:
import logging


n_sentences = 0
n_tokens = 0
n_docs = 0

files = list(SG_DIR.glob(SG_GLOB))


for f in tqdm(files, unit="file", position=0):
    df = pd.read_csv(
        f, compression="gzip", sep=";", encoding=SG_ENCODING, parse_dates=["date"]
    ).dropna(subset="Content")

    for index, value in tqdm(df.Content.items(), unit="doc", total=len(df), position=1):
        if len(value) > 1000000:
            logging.warning(
                f"Skipping long text ({len(value)} characters) in row {index}, file '{f}'."
            )
            continue

        n_docs += 1
        doc = nlp(value)
        for sent in doc.sents:
            n_sentences += 1
            for token in sent:
                n_tokens += 1

print(f"Number of documents:\t{n_docs}")
print(f"Number of sentences:\t{n_sentences}")
print(f"Number of tokens:\t{n_tokens}")


100%|██████████| 60/60 [00:03<00:00, 15.45doc/s]e]
0doc [00:00, ?doc/s]205 [12:18<28:05, 10.67s/file]
100%|██████████| 52/52 [00:00<00:00, 62.08doc/s]e]
0doc [00:00, ?doc/s]205 [21:08<24:40, 11.84s/file]
100%|██████████| 77/77 [00:01<00:00, 70.20doc/s]63s/file] 
0doc [00:00, ?doc/s]/205 [16:50:39<5:56:04, 237.39s/file]
100%|██████████| 119/119 [00:02<00:00, 46.72doc/s]ile]
0doc [00:00, ?doc/s]/205 [16:55:59<13:28, 10.78s/file]
100%|██████████| 205/205 [17:11:27<00:00, 301.89s/file]

Number of documents:	: 187133
Number of sentences:	: 47302585
Number of tokens:	: 1167766456
